# embeddings based on local model, feedforward neural network

In [27]:
# imnport libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

# display CUDA devices available
torch.cuda.get_device_name(0)

# set device to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [48]:
# create model
import torch


class FeedForward(torch.nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(12 * 8 * 8 + 2, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, 256)  # 2 for turn
        self.fc4 = torch.nn.Linear(256, 128)  # 2 for turn
        self.fc5 = torch.nn.Linear(128, 64)
        self.fc6 = torch.nn.Linear(64, 1)

        self.activation = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.25)

    def forward(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc6(x)

        return x

    def predict(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.fc6(x)

        return x

    def embed(self, x):
        # reshape x to be 12*8*8
        # x = x.view(-1, 12*8*8)

        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)

        return x



In [49]:
# load data

evals = np.load('../../data/evaluations.npy')
turns = np.load('../../data/turns.npy')
one_hot_boards = np.load('../../data/one_hot_boards.npy')

# reshape data to be 2D
evals = evals.reshape(-1, 1)
# normalize data to be / 100
EVAL_DIVISOR = 1000
evals = evals / EVAL_DIVISOR

# shuffle data
np.random.seed(0)
np.random.shuffle(evals)
np.random.seed(0)
np.random.shuffle(turns)
np.random.seed(0)
np.random.shuffle(one_hot_boards)

# split data into train and test
SPLIT_INDEX = int(len(evals) * 0.8)
train_evals = evals[:SPLIT_INDEX]
train_turns = turns[:SPLIT_INDEX]
train_boards = one_hot_boards[:SPLIT_INDEX]

test_evals = evals[SPLIT_INDEX:]
test_turns = turns[SPLIT_INDEX:]
test_boards = one_hot_boards[SPLIT_INDEX:]

# convert to tensors
train_evals = torch.from_numpy(train_evals).float()
train_turns = torch.from_numpy(train_turns).float()
train_boards = torch.from_numpy(train_boards).float()

test_evals = torch.from_numpy(test_evals).float()
test_turns = torch.from_numpy(test_turns).float()
test_boards = torch.from_numpy(test_boards).float()

# flatten boards and add turns 
train_boards = train_boards.view(train_boards.shape[0], -1)
train_boards = torch.cat((train_boards, train_turns), 1)

test_boards = test_boards.view(test_boards.shape[0], -1)
test_boards = torch.cat((test_boards, test_turns), 1)

In [50]:
# load model
model = FeedForward().to(device)
model.load_state_dict(torch.load('model.pth'))
loss = torch.nn.L1Loss()


# test model
y_pred = model.predict(test_boards.to(device))
test_loss = loss(y_pred, test_evals.to(device)).item()*EVAL_DIVISOR
print('Test loss: ', test_loss)

Test loss:  90.45369923114777


In [55]:
# embed boards

# embed train boards
train_embeds = model.embed(train_boards.to(device))

# embed test boards
print(train_embeds)

tensor([[-0.1001, -0.1295, -0.1408,  ...,  0.2095,  0.2381, -0.1323],
        [ 0.9063,  0.4279,  0.4771,  ..., -0.4030, -0.6774,  0.4595],
        [ 0.0542,  0.0651,  0.0818,  ..., -0.0555, -0.0535,  0.0905],
        ...,
        [ 0.5806,  0.2923,  0.3271,  ..., -0.2770, -0.4401,  0.3226],
        [ 0.4570,  0.2378,  0.2688,  ..., -0.2125, -0.3220,  0.2680],
        [ 0.0156,  0.0445,  0.0556,  ..., -0.0314, -0.0193,  0.0683]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [64]:
# import libraries
# TSNE embedding

from sklearn.manifold import TSNE
import seaborn as sns

# create TSNE embedding
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=800)

# fit TSNE embedding and plot in xyz
tsne_results = tsne.fit_transform(train_embeds.cpu().detach().numpy())

# plot TSNE embedding


# show plot
plt.show()



[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 3881 samples in 0.000s...
[t-SNE] Computed neighbors for 3881 samples in 0.183s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3881
[t-SNE] Computed conditional probabilities for sample 2000 / 3881
[t-SNE] Computed conditional probabilities for sample 3000 / 3881
[t-SNE] Computed conditional probabilities for sample 3881 / 3881
[t-SNE] Mean sigma: 0.015530
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.746323
[t-SNE] KL divergence after 800 iterations: 0.287405


AttributeError: 'PathCollection' object has no property 'z'